# Example Notebook For Group Projects#

08-28-2015

This is an example notebook for the types of analysis that will be performed on Day 5 of the Biocomputing Bootcamp.

#### Setup

In [ ]:
# imports matplotlib
# import pyplot to plt
%pylab inline
import numpy as np
import subprocess
import re
from scipy import stats
import os
import csv
import vcf
import sys
import pysam
from operator import itemgetter
from itertools import groupby, cycle

workDir ='/scratch/biobootcamp_fluxod/myname/day5/'
genoDir= '/scratch/biobootcamp_fluxod/kitzmanj/geuvadis/genotypes_fixed/'
expDir = '/scratch/biobootcamp_fluxod/remills/bioboot/geuvadis/analysis_results'

os.chdir(workDir)

print 'Work directory is',os.getcwd()
print 'Genotype directory is',genoDir
print 'Gene Expression directory is',expDir

In [ ]:
# you will hit an error in the above line.
# read the error message and determine what needs to be done (here and at the terminal)

###Obtaining Data
Scientists must often make use of large data sets that are made available on either websites or data repositories. Here, we have already downloaded the pertinent data, but for reference here is a way the data could be obtained.

In [ ]:

#os.chdir(expDir)
#subprocess.call(["wget", "ftp://ftp.ebi.ac.uk/pub/databases/microarray/data/experiment/GEUV/E-GEUV-1/analysis_results/*"])
#os.chdir(genoDir)
#subprocess.call(["wget", "ftp://ftp.ebi.ac.uk/pub/databases/microarray/data/experiment/GEUV/E-GEUV-1/genotypes/*"])
#os.chdir(workDir)

### Parsing the expression value table
The file with gene-level quantification under the data directory
in a file named GeneQuantRPKM.txt

This file is a matrix of gene expression values across genes (rows) vs individuals (columns)

We'd like to parse the following:
  1. List of individual/sample names
  2. List of gene names
  
  
Go back to the shell and check it out via unix commands (less, cut, head)

In [ ]:
pathToQuantFile = expDir + '/GD660.GeneQuantRPKM.txt'

#### Get list of individual/sample names from the quant file

 - first open the file for reading
 - Loop through the file, line by line
 - At each line, extract the gene name and add it to a list

In [ ]:
def readIndivNamesFromExpressionTable( tableFileName ):
    tableFile = open( tableFileName, 'r' )
    
    firstLine = tableFile.readline()
    # split at the tab separators into a list
    firstLine = firstLine.rstrip().split('\t')
    
    firstLine = [ name.split('.')[0] for name in firstLine ]
    
    # everything after the fourth column
    return firstLine[4:]

In [ ]:
expressionTableIndivs = readIndivNamesFromExpressionTable( pathToQuantFile )

# this returns a dict mapping from person name to the column number in the file

In [ ]:
# sanity check - is this right?  (go back to your flux terminal and verify)
expressionTableIndivs.index( 'HG00105' )

#### DIY - get list of gene names from the quant file

 - first open the file for reading
 - Loop through the file, line by line
 - At each line, extract the gene name and add it to a list
 - final list should have entries of the form "ENSG###########" 


In [ ]:
def readGeneNamesFromExpressionTable( tableFileName ):
    tableFile = open( tableFileName, 'r' )
    
    # ...

    # return something
    

In [ ]:
listOfGeneNames = readGeneNamesFromExpressionTable( pathToQuantFile )

print listOfGeneNames[0:10]

#### DIY - extract a given gene by name from this file

 - first open the file for reading
 - Loop through the file, line by line
 - If the gene name is equal to the one we're looking for, process the line:
     - extract numerical expression values from the line
     - make a dictionary from individual name to expression value
     - then return the dictionary
   
   
 - advanced: to perform this operation much more quickly, explore using pandas
 <pre>
 import pandas as pd
 tbl = pd.read_csv(filename, sep='\t')
 tbl = tbl.set_index #  (....a column name - which one do you want to index on? )
 tbl.ix['ENSG00000215372.5']



In [ ]:
def extractExpressionValues( tableFileName, geneName, indivList  ):

    # open the file
    
    # loop over lines
    
    # when you reach a matching line, put expression value and individual name into a dictionary
    
    # hint 1: if you are part of the way there but stuck, try putting print statements 
    #    in your code to debug
    # 
    # hint 2: there many ways to do this -- but one useful function is zip
    #
    #   https://docs.python.org/2/library/functions.html#zip
    #   
    #   test combining two lists 
    #       my_favorites = zip( ['brewery','ice cream','peninsula'], ['new glarus','washtenaw','olympic'] )
    #   
    #   this will make a list of tuples
    #    
    #   a dictionary can be made from a list of tuples by calling
    #     dict( my_favorites ) 
    #  
    # hint 3: if you have a list that you need to convert to floating point numbers
    #    you could use a list comprehension such as
    #    one_list = [ float(thing) for thing in another_list ]
    #    
    
    # return ...
    
    return



In [ ]:
indivToExpr = extractExpressionValues( pathToQuantFile, 'ENSG00000151503', expressionTableIndivs )

In [ ]:
print indivToExpr.keys()[:10]
print indivToExpr.values()[:10]

#### DIY:  visualize the distribution of expression values for this gene

- refer back the day3 basic plotting notebook (https://github.com/bioboot/web-2015/blob/gh-pages/class-material/basic_matplotlib_plots.ipynb) to plot rank-ordered values


### Pick a region

chr5:95,984,676-96,185,176 - Alcohol Dependence - ERAP1 - rs13160562 GWAS, rs1057569 eQTL  
chr11:47,426,802-47,627,302 - Body Mass Index - MTCH2 - rs3817334 GWAS, rs10838724 eQTL  
chr1:205,620,478-205,820,978 - Parkinsons disease - RAB7L1 - rs947211 GWAS, rs708725 eQTL  
chr21:38,432,812-38,633,312 - Eye color traits - TTC3 - rs1003719 GWAS, rs3787788 eQTL  
chr20:5,883,504-6,084,004 - HIV-1 control - MCM8 - rs454422 GWAS, rs13041190 eQTL  
chr5:156,807,522-157,008,022 - Pulmonary function - ADAM19 - rs2277027 GWAS, rs9313615 eQTL 

Go to ENSEMBL and find the gene identifier (starts "ENSG..") for your gene symbol (ERAP1, MTCH2,...)

Under "Search:" select Human, and enter your gene name.  You'll see a list of results, and look 
for an identifier beginning with "ENSG"

Quickly check for that line in the table using grep.  

In [ ]:
chromo = '5'
startPos, endPos = 95984676, 96185176

#### DIY - extract & plot expression values for your gene
 - plot as a ranked list 
 - plot as a histogram
 - plot as a boxplot

### Extract genotype data
Now that we have the files in the proper format, we will pull out the relevant portions to 
our analysis using one of the regions of interest. 

Given that each region is ~200,000 bp in size, 
there will likely be many more SNPs than there are genes in the region. 

 - In your flux terminal, check out the genotypes directory.  Which file do you need to use?
 - These files are large!  We will not loop through the entire chromosome vcf just for our small region
 - Instead, we can use tabix -- try this from the command line and extract the results into an individual VCF in your own directory
 - use bgzip and tabix to compress this file


In [ ]:
pathToMyRegionGenotypes = workDir + '/ERAP1_flank200k_genotypes.vcf.gz'

#### Try out PyVCF 

Quick introduction: http://pyvcf.readthedocs.org/en/latest/INTRO.html 

This is a useful python library for reading VCF files, so you don't have to write your own parser

In [ ]:
vcfFile = vcf.Reader( open('/scratch/biobootcamp_fluxod/remills/bioboot/geuvadis/genotypes/GEUVADIS.chr9.PH1PH2_465.IMPFRQFILT_BIALLELIC_PH.annotv2.genotypes.vcf.gz','r') )

In [ ]:
line = vcfFile.next()

In [ ]:
print line

In [ ]:
print line.genotype( 'NA07357' )

In [ ]:
print line.INFO

In [ ]:
# what is the allele frequency (overall among our samples) of this variant?
print line.INFO['AF']
# what would 0.01 mean?  
# what would 0.99 mean?

In [ ]:
print line.genotype( 'NA07357' )['GT']

In [ ]:
#  http://pyvcf.readthedocs.org/en/latest/API.html?highlight=gt_type#vcf.model._Call.gt_type
print line.genotype( 'NA07357' ).gt_type

#### Select rows by genomic coordintate 

PyVCF has tabix functionality 'baked in'

Use the .fetch() command

In [ ]:
# count number of variant records on chrom 9 between 40 and 40.1 Mbp
countOfLines = 0
for line in vcfFile.fetch( '9', 40000000, 40100000 ):
    countOfLines += 1
    
print countOfLines

#### DIY- count number of rows in same region with MAF > 5%

#### DIY- count number of rows YOUR region with MAF > 5%

In [ ]:
print chromo,startPos,endPos

### Do a ordinary linear regression in python

- Using the function stats.linrgress 
- Consult documentation here http://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.linregress.html
- Or, use shift-TAB (as on Day 3) to get a brief description of arguments



#### Test with some well-correlated random samples
- sample X  ~ N(10,2)
- sample Y ~ 3*x + e, e ~ N(0,1)
- plot the points X versus Y
- plot the best fit line 



In [ ]:
# ...

print slope
print intercept
print r_value
print p_value
print std_err


In [ ]:
# DIY: plot x vs y

In [ ]:
# DIY: plot x versus y and best fit line

### Put it all together

- Loop through a list of genes in the region
- Get gene expression values
- In that region, loop through every SNP
    - Only consider SNPs with MAF > 5%
- For each SNP, perform a linear regression of expression against genotype (0,1,2)

In [ ]:
#####

In [ ]:
#
# define a function that will take in:
#    fnQuant - filename of the gene expression file
#    fnRegionVcf - filename of the genotypes (.vcf.gz)
#    gene - identifier of the gene for eQTL analysis 
#    chromo - chromosome name of the eQTL analysis region (doesn't necessarily have to overlap gene but usually does)
#    startPos - starting position of eQTL analysis region
#    endPos - ending position of eQTL analysis region
#    expressoinTableIndivs - list of individuals in the expression table
#    
def regionVsSingleGene( fnQuant,
                        fnRegionVcf,
                        gene, 
                        chromo,
                        startPos, 
                        endPos,
                        expressionTableIndivs,
                        cutoffToPlot = 1e-40,
                        makePlots=True ):

    
    
    
    
    
    
    #
    # do some stuff .... 
    #
    
    # 
    # scroll down for hint/outline
    # 
    
    
    
    
    
    
    
    
    
    
    
    
    
    return bestPval, bestSnp, listOfPvalues, listOfSnpCoords
            

In [ ]:
#
# define a function that will take in:
#    fnQuant - filename of the gene expression file
#    fnRegionVcf - filename of the genotypes (.vcf.gz)
#    gene - identifier of the gene for eQTL analysis 
#    chromo - chromosome name of the eQTL analysis region (doesn't necessarily have to overlap gene but usually does)
#    startPos - starting position of eQTL analysis region
#    endPos - ending position of eQTL analysis region
#    expressoinTableIndivs - list of individuals in the expression table
#    
def regionVsSingleGene( fnQuant,
                        fnRegionVcf,
                        gene, 
                        chromo,
                        startPos, 
                        endPos,
                        expressionTableIndivs,
                        cutoffToPlot = 1e-40,
                        makePlots=True ):

    # do some setup - open the genotypes
    vcfFile = vcf.Reader( open(fnRegionVcf,'r') )
    
    # DIY - use our previously defined function readIndivNamesFromExpressionTable
    # to load the names of individuals from the gene expression table
    
    # DIY - use our previously defined function extractExpressionValues
    # to load the epxression values of this gene into a dictionary
    
    # DIY - convert the expression values to a list 
    #listOfExpressionValues = []
    #for sample in #something#:
    #    listOfExpressionValues.append( #SOMETHING... )
    
    # setup some tracking values
    bestPval = 1e99        #  best P-value so far 
    bestSnp = None         # best SNP position so far 
    listOfPvalues = []     # list of per-SNP P-values 
    listOfSnpCoords = []   # list of per-SNP coordinates
    counter = 0            # increment this every time we go through the vcf file

    # now query the vcf file for variants 
    
    for record in vcfFile.fetch( chromo, startPos, endPos ):
    
        # 
        counter += 1
        if counter % 250 == 0 :
            print 'on snp #',counter,'...'

        #  DIY - if this SNP minor AF is < 5%, skip it
        #
        
        # DIY append the current SNP coordinate to that list. 

        listOfSnpCoords.append( record.POS )
        
        # DIY - make a list of genotype calls,
        #  in the same order as the gene expression and individual name
        #  lists
        #listOfGenotypeCalls = []

        #for sample in SOMETHING: 
            #
            #get the genotype record for this current sample
            #
            #use the .gt_type as we did above to extract a genotype (0,1,2) 
            #and append this to listOfGenotypeCalls.
            #
            # before appending, check that geno is 0 or 1 or 2 ... if not, just append 0
            #

            
        # DIY - now we have a list of genotypes as numbers and a list of 
        #   expression values.  Convert each to a numpy array, e.g.,   something = np.array(something_else)

        # DIY - perform linear regression for *this marker's* genotypes vs 
        #  the gene's expression level
        #
        #  append the p value;  if it's less than the current best p-value, 
        #    then update the current best p value and location
        
        # DIY - scatter plot with regression 
        #if p_value < cutoffToPlot and makePlots:
            print record.ID
            print 'gene',gene
            print 'r value', r_value
            print  'p_value', p_value
            print 'standard deviation', std_err

            # plot expression level vs genotype as a scatter plot
            # plot the regression line 
            
    return bestPval, bestSnp, listOfPvalues, listOfSnpCoords
            

In [ ]:
# now we'll use this function

bestPval, bestSnp, listOfPvalues, listOfSnpCoords = regionVsSingleGene( 
        pathToQuantFile,
        pathToMyRegionGenotypes,
        'ENSG00000164307', 
        chromo,
        startPos, 
        endPos,
        cutoffToPlot=1e-65,
        makePlots=True )

In [ ]:
print 'best p value is',bestPval

#### DIY: modify above code to produce a boxplot rather than a scatterplot
- hint - in addition to making a list of all x (genotype) and y (expression values), also construct lists of values for the 0/0, 0/1, and 1/1 genotypes

In [ ]:
def regionVsSingleGeneBoxplot( fnQuant,
                        fnRegionVcf,
                        gene, 
                        chromo,
                        startPos, 
                        endPos,
                        cutoffToPlot = 1e-40,
                        makePlots=True ):

    # copy above function and make a few modifications

In [ ]:
# now we'll use this function
bestPval, bestSnp, listOfPvalues, listOfSnpCoords = regionVsSingleGeneBoxplot( 
        pathToQuantFile,
        pathToMyRegionGenotypes,
        'ENSG00000164307', 
        chromo,
        startPos, 
        endPos,
        cutoffToPlot=1e-65,
        makePlots=True )

#### DIY: make a Manhattan plot
 - SNP p-value versus coordinate
 - usually it's best to view these as -log10(p) -- numpy has a log10 function

### Challenge ideas
 - Expand the regions, then loop over multiple genes within the region.  Do cis-eQTLs for a given gene act as the same for adjacent genes?
 - Implement a permutation method to estimate false discovery rate.  For instance, you could add a new function to scramble the genotype values amongst individuals as they are read (or likewise for the expression values)